In [19]:
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline


df = pd.read_csv('../assets/data/user_info.csv')

from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')
driver = webdriver.Chrome("../assets/chromedriver", options=options)

df = df.drop('username',axis=1)

kd = [i/(1+sum([df.qps_elims,df.qps_deaths])) for i in [df.qps_elims,df.qps_deaths]]
df['kill_ratio'] = kd[0]
df['deaths_ratio'] = kd[1]

def get_team():
    index = [i for i in np.random.randint(0,df.shape[0],size=12)]
    
    column0 = []
    column1 = []
    for col in df.columns:
        column0.append(col+str(0))
        column1.append(col+str(1))
    
    team1 = df.iloc[index[0:6]].mean(axis=0)
    team2 = df.iloc[index[6:12]].mean(axis=0)
    
    t1 = 0
    t2 = 0
    for col in df.columns:
        if 'deaths' in col:
            if team1[col] > team2[col]:
                t1 = t1 - 1
                t2 = t2 + 1
            else:
                t1 = t1 + 1
                t2 = t2 - 1
        else:
            if team1[col] > team2[col]:
                t1 = t1 + 1
                t2 = t2 - 1
            else:
                t1 = t1 - 1
                t2 = t2 + 1
    
    data = dict(zip(column0,team1))
    data1 = dict(zip(column1,team2))
    
    data2 = pd.DataFrame([data,data1])
    data3 = pd.DataFrame(data2.max()).T
    
    if np.random.randint(0,100) >= 90:
        t1 = t1 + 10
    elif np.random.randint(0,100) <= 10:
        t2 = t2 + 10
    
    if t1 > t2:
        data3['won'] = 0
    elif t2 > t1:
        data3['won'] = 1
    else:
        data3['won'] = 0
        
    return data3


team_averages = pd.DataFrame()
for i in range(1000):
    team_averages = team_averages.append(get_team())


train, test = train_test_split(team_averages, train_size=0.80, test_size=0.20, 
                              stratify=team_averages['won'], random_state=42)

target = 'won'
X_train = train.drop(columns=target)
y_train = train[target]
X_test = test.drop(columns=target)
y_test = test[target]
train.shape, test.shape

# target = 'won'
# X_train = team_averages.drop(columns=target)
# y_train = team_averages[target]

pipeline = make_pipeline(
    RandomForestClassifier(n_estimators=1000,max_depth=20)
)

# Fit on train, score on val
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('randomforestclassifier',
                 RandomForestClassifier(bootstrap=True, class_weight=None,
                                        criterion='gini', max_depth=20,
                                        max_features='auto',
                                        max_leaf_nodes=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=1000, n_jobs=None,
                                        oob_score=False, random_state=None,
                                        verbose=0, warm_start=False))],
         verbose=False)

In [20]:
team_averages.head()

,level0,rating0,prestige0,games_won0,qps_elims0,qps_dmg_done0,qps_deaths0,qps_solo_kills0,medals_bronze0,medals_silver0,...,qps_elims1,qps_dmg_done1,qps_deaths1,qps_solo_kills1,medals_bronze1,medals_silver1,medals_gold1,kill_ratio1,deaths_ratio1,won
0,36.500000,752.833333,2.500000,371.833333,8578.833333,1.855499e+06,4647.500000,1276.833333,580.333333,625.166667,...,9007.5,2.175759e+06,5405.000000,1437.666667,587.166667,591.000000,587.333333,0.678720,0.320889,1
0,38.833333,993.833333,4.666667,658.666667,14785.166667,2.877288e+06,8343.333333,1763.833333,897.666667,1046.500000,...,1980.5,5.496302e+05,743.500000,410.166667,96.000000,117.666667,159.833333,0.713857,0.284858,0
0,48.333333,606.333333,2.500000,544.000000,13003.000000,2.194088e+06,6742.333333,1670.666667,759.833333,935.500000,...,3966.0,7.482490e+05,1825.666667,707.500000,214.666667,256.166667,321.333333,0.671694,0.327984,0
0,44.333333,285.500000,1.833333,214.166667,4486.333333,9.717895e+05,2486.000000,725.500000,289.666667,309.666667,...,5507.0,8.023833e+05,2746.666667,1183.833333,340.333333,393.666667,419.666667,0.671959,0.327860,1
0,40.833333,365.166667,2.833333,482.166667,12065.166667,1.672060e+06,5732.166667,2438.666667,746.000000,785.500000,...,8308.0,1.083988e+06,4372.666667,1587.166667,477.166667,527.833333,606.666667,0.667039,0.332704,0


In [21]:
rfc_y_pred = pipeline.predict(X_test)
print('Accuracy Score', accuracy_score(y_test, rfc_y_pred))

Accuracy Score 0.925


In [22]:
import sklearn
import pickle
with open('pipeline.pkl', 'wb') as f:
    pickle.dump(pipeline, f)

In [ ]:
from joblib import dump, load
dump(pipeline, 'pipeline.joblib')

In [ ]:
pipeline = load('pipeline.pc')

In [23]:
class Player:
    def __init__(self, name, level, rating, prestige, games_won, qps, medals):
        self.name = name
        self.level = level
        self.rating = rating
        self.prestige = prestige
        self.qps = qps
        self.medals = medals
        self.games_won = games_won

class Stats:
    def __init__(self, elims=0, dmg_done=0, deaths=0, solo_kills=0):
        self.elims = elims
        self.dmg_done = dmg_done
        self.deaths = deaths
        self.solo_kills = solo_kills
        
class Medals:
    def __init__(self, bronze=0, silver=0, gold=0):
        self.bronze = bronze
        self.silver = silver
        self.gold = gold

def create_player(js):
    if 'quickPlayStats' not in js:
        return Player(js['name'], js['level'],js['rating'],js['prestige'], 0, Stats(), Medals())
    if 'careerStats' not in js['quickPlayStats']:
        return Player(js['name'], js['level'],js['rating'],js['prestige'], 0, Stats(), Medals())
    if js.get('quickPlayStats',{}).get('careerStats',{}) == None or 'allHeroes' not in js.get('quickPlayStats',{}).get('careerStats',{}):
        return Player(js['name'], js['level'],js['rating'],js['prestige'], 0, Stats(), Medals())
    
    elims = 0
    damageDone = 0
    deaths = 0
    soloKills = 0

    if js['quickPlayStats']['careerStats']['allHeroes']['combat'] != None:

        if 'eliminations' in js['quickPlayStats']['careerStats']['allHeroes']['combat']:
            elims = js['quickPlayStats']['careerStats']['allHeroes']['combat']['eliminations']

        if 'damageDone' in js['quickPlayStats']['careerStats']['allHeroes']['combat']:
            damageDone = js['quickPlayStats']['careerStats']['allHeroes']['combat']['damageDone']

        if 'deaths' in js['quickPlayStats']['careerStats']['allHeroes']['combat']:
            deaths = js['quickPlayStats']['careerStats']['allHeroes']['combat']['deaths']

        if 'soloKills' in js['quickPlayStats']['careerStats']['allHeroes']['combat']:
            soloKills = js['quickPlayStats']['careerStats']['allHeroes']['combat']['soloKills']
    
    qps = Stats(elims,damageDone,deaths,soloKills)

    medals = Medals(js['quickPlayStats']['awards'].get('medalsBronze'),
                      js['quickPlayStats']['awards'].get('medalsSilver'),
                      js['quickPlayStats']['awards'].get('medalsGold'))
    
    return Player(js['name'], js['level'],js['rating'],js['prestige'], js['quickPlayStats']['games']['won'], qps, medals)

def df_object(p):
    item = [p.name,p.level,p.rating,p.prestige,p.games_won,p.qps.elims,p.qps.dmg_done,
            p.qps.deaths,p.qps.solo_kills,p.medals.bronze,p.medals.silver,p.medals.gold]
        
    return item

def select_player(username):
    url = f"https://ow-api.com/v1/stats/pc/us/{username}/complete"
    print(url)
    response = requests.get(url)
    j = json.loads(response.text)
    return create_player(j)

##dataframe setup
columns = ['username','level','rating','prestige','games_won','qps_elims','qps_dmg_done',
           'qps_deaths','qps_solo_kills','medals_bronze','medals_silver','medals_gold']

def create_teams(*args):
    team1 = []
    team2 = []
    teams_dataframe = pd.DataFrame(columns=columns)
    for i in range(len(args)):
        player = select_player(args[i])
        teams_dataframe.loc[len(teams_dataframe), :] = df_object(player)
    
    return teams_dataframe

df = create_teams('Cat-14790','Cat-14790','Cat-23740','Cat-14790','Cat-14790','Cat-14790',
                 'Cat-14790','Cat-14790','Cat-14790','Cat-14790','Cat-14790','Cat-14790')

https://ow-api.com/v1/stats/pc/us/Cat-14790/complete
https://ow-api.com/v1/stats/pc/us/Cat-14790/complete
https://ow-api.com/v1/stats/pc/us/Cat-23740/complete
https://ow-api.com/v1/stats/pc/us/Cat-14790/complete
https://ow-api.com/v1/stats/pc/us/Cat-14790/complete
https://ow-api.com/v1/stats/pc/us/Cat-14790/complete
https://ow-api.com/v1/stats/pc/us/Cat-14790/complete
https://ow-api.com/v1/stats/pc/us/Cat-14790/complete
https://ow-api.com/v1/stats/pc/us/Cat-14790/complete
https://ow-api.com/v1/stats/pc/us/Cat-14790/complete
https://ow-api.com/v1/stats/pc/us/Cat-14790/complete
https://ow-api.com/v1/stats/pc/us/Cat-14790/complete


In [41]:
def predict(data): 
    data = data.drop('username',axis=1)

    kd = [i/(1+sum([data.qps_elims,data.qps_deaths])) for i in [data.qps_elims,data.qps_deaths]]
    data['kill_ratio'] = kd[0]
    data['death_ratio'] = kd[1]
    
    column0 = []
    column1 = []
    for col in data.columns:
        column0.append(col+str(0))
        column1.append(col+str(1))
    
    team1 = data.iloc[0:6].mean(axis=0)
    team2 = data.iloc[6:12].mean(axis=0)
    
    t1 = 0
    t2 = 0
    for col in data.columns:
        if 'deaths' in col:
            if team1[col] > team2[col]:
                t1 = t1 - 1
                t2 = t2 + 1
            else:
                t1 = t1 + 1
                t2 = t2 - 1
        else:
            if team1[col] > team2[col]:
                t1 = t1 + 1
                t2 = t2 - 1
            else:
                t1 = t1 - 1
                t2 = t2 + 1
    
    data1 = dict(zip(column0,team1))
    data2 = dict(zip(column1,team2))
    
    data3 = pd.DataFrame([data1,data2])
    data4 = pd.DataFrame(data3.max()).T
    
    if np.random.randint(0,100) >= 90:
        t1 = t1 + 10
    elif np.random.randint(0,100) <= 10:
        t2 = t2 + 10
    
    if t1 > t2:
        data4['won'] = 0
    elif t2 > t1:
        data4['won'] = 1
    else:
        data4['won'] = 0
        
    return data4

team_averages = predict(df)

team_averages.head()

,level0,rating0,prestige0,games_won0,qps_elims0,qps_dmg_done0,qps_deaths0,qps_solo_kills0,medals_bronze0,medals_silver0,...,qps_elims1,qps_dmg_done1,qps_deaths1,qps_solo_kills1,medals_bronze1,medals_silver1,medals_gold1,kill_ratio1,death_ratio1,won
0,25.833333,317.0,1.166667,103.5,1962.666667,564895.5,1409.333333,249.833333,139.833333,139.333333,...,1926.0,570311.0,1126.0,276.0,131.0,117.0,103.0,0.630855,0.368818,0


In [42]:
X_test = team_averages.drop(columns=target)
y_pred = pipeline.predict(X_test)
y_pred

array([1], dtype=int64)

In [34]:
print('Accuracy Score', accuracy_score(team_averages['won'], y_pred))
# y_pred,X_test

Accuracy Score 0.0


In [51]:
url = 'https://ow-api.com/v1/stats/pc/us/Panda-29528/complete'
print(url)
response = requests.get(url)
j = json.loads(response.text)
if(j['error']):
    print('nope')
else:
    create_player(j)

https://ow-api.com/v1/stats/pc/us/Panda-29528/complete
nope
